In [1]:
from tensorflow import keras

In [2]:
arkModel = keras.models.load_model('C:/MLMODEL')

In [3]:
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
import numpy as np

In [4]:
import math

In [5]:
labels = ['apple','bag', 'cap', 'doll', 'elephant', 'fox', 'giraffe', 'house', 'icecream', 'jamjar', 'kite', 'ladybug', 'monkey', 'nest', 'orange', 'parrot', 'queen', 'rainbow', 'ship', 'tiger', 'umbrella', 'van', 'whale', 'xylophone', 'yoyo', 'zebra', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten']

In [6]:
import queue
result_queue = queue.Queue()

def putLabel(img_path):
    img = image.load_img(img_path, target_size=(224,224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    
    result = arkModel.predict(x)
    ans = result[0]

    max_acc = 0
    max_index = 0

    for i in range(36):
        #acc = f'{ans[i]*100:.20f}'
        if ans[i]>max_acc:
            max_acc = ans[i]
            max_index = i
        #print(acc)

    result_queue.put(labels[max_index]+":"+str(round(max_acc*100, 2)))
#     prediction_result = result_queue[-1]
#     print('[#] Prediction result: ', prediction_result, '%')

In [7]:
import socket
import base64

HOST = '172.16.49.223'  # Standard loopback interface address (localhost)
PORT = 7919        # Port to listen on (non-privileged ports are > 1023)
RES_SENDER_PORT = 7921 # Port to listen on for label requests

In [8]:
import threading

In [9]:
def listener_thread():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((HOST, PORT))
        s.listen()

        print('[+] \033[1mARK Deep Learning & Computer Vision Server\033[0m [+]')
        print('[~] ============= Status: Active ============= [~]')
        print('[-] ========== Waiting for requests ========== [-]')

        while True:
            conn, addr = s.accept()

            base64_image_string = ""

            with conn:
                print('[i] Request received from: ', addr)
                while True:
                    try:
                        data = conn.recv(1024)
                        base64_image_string += data.decode('utf-8')

                    except Exception as e:
                        break
                    if not data:
                        break

                #print(base64_image_string)
                with open("received_image", "wb") as fh:
                    fh.write(base64.decodebytes(str.encode(base64_image_string)))

                putLabel("received_image")

In [10]:
def result_sender_thread():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((HOST, RES_SENDER_PORT))
        s.listen()

        while True:
            conn, addr = s.accept()
            with conn:
                label = result_queue.get()
                conn.send(label.encode('utf-8'))

In [11]:
listener_thread_instance = threading.Thread(target=listener_thread, args=())
sender_thread_instance = threading.Thread(target=result_sender_thread, args=())

In [ ]:
listener_thread_instance.start()
sender_thread_instance.start()

listener_thread_instance.join()
sender_thread_instance.join()

[+] ARK Deep Learning & Computer Vision Server [+]
[~] ============= Status: Active ============= [~]
[-] ========== Waiting for requests ========== [-]
[i] Request received from:  ('172.16.49.203', 43888)
[i] Request received from:  ('172.16.49.203', 43896)
[i] Request received from:  ('172.16.49.203', 43902)
[i] Request received from:  ('172.16.49.203', 43908)
[i] Request received from:  ('172.16.49.203', 43914)
[i] Request received from:  ('172.16.49.203', 43918)
[i] Request received from:  ('172.16.49.203', 43922)
[i] Request received from:  ('172.16.49.203', 43928)
